In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import maup
import json
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry.polygon import orient

# Import dataframes, dictionaries, and functions

In [2]:
economic_columns = [
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
    '60K-100K', '100K_125K', '125K_150K', '150K_MORE'
]
race_columns = [
    'TOT_POP22', 'NHSP_POP22', 'HSP_POP22', 
    'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 
    'ASN_NHSP22', 'HPI_NHSP22', 'OTH_NHSP22'
]

In [3]:
def orient_geometry(geom):
    """
    Args:
        geom: A Shapely geometry object (Polygon or MultiPolygon).
    
    Returns:
        A reoriented geometry object. If the input geometry is neither a Polygon
        nor a MultiPolygon, it is returned unchanged.
    """
    if geom.type == 'Polygon':
        return orient(geom, sign=1.0)
    elif geom.type == 'MultiPolygon':
        return MultiPolygon([orient(part, sign=1.0) for part in geom.geoms])
    else:
        return geom

In [4]:
def aggregate_economic_data(block_gdf, precincts_gdf, variables):
    """
    Parameters:
    - block_gdf (GeoDataFrame): GeoDataFrame containing the economic data per census block
    - precincts_gdf (GeoDataFrame): GeoDataFrame representing precincts to which data will be aggregated
    - variables (list): List of column names to be aggregated.

    Returns:
    - GeoDataFrame: The updated precincts_gdf with aggregated economic data.
    """
    precincts_gdf['geometry'] =  precincts_gdf['geometry'].apply(orient_geometry)
    precincts_gdf =  precincts_gdf.to_crs(epsg=4326)
    block_gdf['geometry'] = block_gdf['geometry'].apply(orient_geometry)
    block_gdf = block_gdf.to_crs(epsg=4326)
    assignment = maup.assign(block_gdf, precincts_gdf)
    
    precincts_gdf[variables] = block_gdf[variables].groupby(assignment).sum()
    
    weighted_sum = (block_gdf['MEDN_INC22'] * block_gdf['TOT_HOUS22']).groupby(assignment).sum()
    total_households = block_gdf['TOT_HOUS22'].groupby(assignment).sum()
    
    precincts_gdf['MEDN_INC22'] = weighted_sum / total_households
    precincts_gdf = precincts_gdf.fillna(0)
    
    return precincts_gdf


In [5]:
def aggregate_racial_data(block_gdf, precincts_gdf, variables):
    """
    Parameters:
    - block_gdf (GeoDataFrame): GeoDataFrame containing the racial data per census block
    - precincts_gdf (GeoDataFrame): GeoDataFrame representing precincts to which data will be aggregated
    - variables (list): List of column names to be aggregated

    Returns:
    - GeoDataFrame: The updated precincts_gdf with aggregated economic data.
    """
    precincts_gdf['geometry'] =  precincts_gdf['geometry'].apply(orient_geometry)
    precincts_gdf =  precincts_gdf.to_crs(epsg=4326)
    block_gdf['geometry'] = block_gdf['geometry'].apply(orient_geometry)
    block_gdf = block_gdf.to_crs(epsg=4326)
    assignment = maup.assign(block_gdf, precincts_gdf)
    
    precincts_gdf[variables] = block_gdf[variables].groupby(assignment).sum()
    
    if 'TOT_POP22' in variables:
        variables.remove('TOT_POP22')
    precincts_gdf['TOT_POP22'] = precincts_gdf[['NHSP_POP22', 'HSP_POP22']].sum(axis=1)
    
    precincts_gdf = precincts_gdf.fillna(0)
    
    return precincts_gdf


In [24]:
def aggregate_region_data(block_gdf, precincts_gdf, variables):
    """
    Parameters:
    - block_gdf (GeoDataFrame): GeoDataFrame containing population data per census block
    - precincts_gdf (GeoDataFrame): GeoDataFrame representing precincts to which data will be aggregated
    - variables (list): List of column names to be aggregated

    Returns:
    - GeoDataFrame: The updated precincts_gdf with aggregated economic and region type data.
    """

    precincts_gdf['geometry'] = precincts_gdf['geometry'].apply(orient_geometry)
    precincts_gdf = precincts_gdf.to_crs(epsg=4326)
    block_gdf['geometry'] = block_gdf['geometry'].apply(orient_geometry)
    block_gdf = block_gdf.to_crs(epsg=4326)

    assignment = maup.assign(block_gdf, precincts_gdf)

    precincts_gdf[variables] = block_gdf[variables].groupby(assignment).sum()
    
    if 'TOT_POP22' in variables:
        variables.remove('TOT_POP22')
    precincts_gdf['TOT_POP22'] = precincts_gdf[['NHSP_POP22', 'HSP_POP22']].sum(axis=1)

    precincts_gdf = precincts_gdf.fillna(0)

    def get_region_type(row):
        if row['TOT_POP22'] >= 5000:
            return 'urban'
        elif 2500 <= row['TOT_POP22'] < 5000:
            return 'suburban'
        else:
            return 'rural'

    precincts_gdf['region_type'] = precincts_gdf.apply(get_region_type, axis=1)

    return precincts_gdf


# Aggregate Census Block data to precincts for South Carolina:

## Get block level geometry

In [6]:
sc_block_inc_gdf = gpd.read_file(
    'raw/census_block/income/sc_inc_2022_bg_shape_file/sc_inc_2022_bg.shp'
)
sc_block_inc_gdf['geometry'] = sc_block_inc_gdf ['geometry'].apply(orient_geometry)
sc_block_inc_gdf = sc_block_inc_gdf .to_crs(epsg=4326)
print(len(sc_block_inc_gdf.columns))

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':


24


In [7]:
sc_block_geometry_gdf = sc_block_inc_gdf[
    ['GEOID','STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry']
]

## Get precincts level geoDataFrames

In [8]:
sc_precincts_gdf = gpd.read_file(
    'states/south_carolina/geodata/south_carolina_precincts.geojson'
)
print(len(sc_precincts_gdf.columns))

6


## Get Census Block categories csv

In [9]:
sc_econ_df = pd.read_csv(
    'processed_individual/sc_econ_block.csv'
)
sc_race_df = pd.read_csv(
    'processed_individual/sc_race_block.csv'
)

## Merging census block geometry with economical data

In [10]:
sc_econ_df = sc_econ_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
sc_block_geometry_gdf['GEOID'] = sc_block_geometry_gdf['GEOID'].astype(str)
sc_econ_df['GEOID'] = sc_econ_df['GEOID'].astype(str)
sc_block_geometry_gdf = sc_block_geometry_gdf.merge(sc_econ_df, on='GEOID')
print(sc_block_geometry_gdf.columns)

Index(['GEOID', 'STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry',
       'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K', '60K-100K', '100K_125K',
       '125K_150K', '150K_MORE'],
      dtype='object')


In [11]:
sc_economic_gdf = sc_block_geometry_gdf[['GEOID', 'geometry'] + economic_columns].copy()
sc_economic_gdf['category'] = 'economic'

In [12]:
sc_block_geometry_gdf['geometry'] = sc_block_geometry_gdf['geometry'].apply(orient_geometry)
final_sc_block_gdf = sc_block_geometry_gdf.to_crs(epsg=4326)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':


In [13]:
print(sc_economic_gdf.columns)

Index(['GEOID', 'geometry', 'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
       '60K-100K', '100K_125K', '125K_150K', '150K_MORE', 'category'],
      dtype='object')


In [14]:
print(sc_economic_gdf.crs)
print(sc_precincts_gdf.crs)

EPSG:4326
EPSG:4326


In [15]:
variables = [
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
    '60K-100K', '100K_125K', '125K_150K', '150K_MORE'
]

In [16]:
sc_precincts_econ_gdf = aggregate_economic_data(sc_economic_gdf, sc_precincts_gdf, variables)


/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute ins

In [17]:
print(sc_precincts_econ_gdf.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'NAME' 'Prec_Code' 'geometry'
 'MEDN_INC22' 'TOT_HOUS22' '0_35K' '35K_60K' '60K-100K' '100K_125K'
 '125K_150K' '150K_MORE']


In [38]:
sc_precincts_econ_gdf = sc_precincts_econ_gdf.drop(columns=['geometry'])
sc_precincts_econ_gdf_json = sc_precincts_econ_gdf.to_json(orient='records')


In [40]:
json_list = json.loads(sc_precincts_econ_gdf_json )

for record in json_list[:5]:
    print(json.dumps(record, indent=4))

{
    "UNIQUE_ID": "ABBEVILLE-:-ABBEVILLE NO. 1",
    "COUNTYFP": "001",
    "County": "ABBEVILLE",
    "NAME": "ABBEVILLE NO. 1",
    "Prec_Code": "001",
    "MEDN_INC22": 67689.0,
    "TOT_HOUS22": 600.0,
    "0_35K": 125.0,
    "35K_60K": 120.0,
    "60K-100K": 166.0,
    "100K_125K": 53.0,
    "125K_150K": 99.0,
    "150K_MORE": 37.0
}
{
    "UNIQUE_ID": "ABBEVILLE-:-ABBEVILLE NO. 2",
    "COUNTYFP": "001",
    "County": "ABBEVILLE",
    "NAME": "ABBEVILLE NO. 2",
    "Prec_Code": "002",
    "MEDN_INC22": 37895.0,
    "TOT_HOUS22": 432.0,
    "0_35K": 204.0,
    "35K_60K": 181.0,
    "60K-100K": 30.0,
    "100K_125K": 0.0,
    "125K_150K": 0.0,
    "150K_MORE": 17.0
}
{
    "UNIQUE_ID": "ABBEVILLE-:-ABBEVILLE NO. 3",
    "COUNTYFP": "001",
    "County": "ABBEVILLE",
    "NAME": "ABBEVILLE NO. 3",
    "Prec_Code": "003",
    "MEDN_INC22": 5199.3483535529,
    "TOT_HOUS22": 577.0,
    "0_35K": 323.0,
    "35K_60K": 39.0,
    "60K-100K": 123.0,
    "100K_125K": 53.0,
    "125K_150K": 

In [42]:
with open('states/south_carolina/economic/south_carolina_precincts_household_income.json', 'w') as json_file:
    json.dump(sc_precincts_econ_gdf_json, json_file, indent=4)

## Merging census block geometry with racial data

In [17]:
sc_race_df = sc_race_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
sc_block_geometry_gdf['GEOID'] = sc_block_geometry_gdf['GEOID'].astype(str)
sc_race_df['GEOID'] = sc_race_df['GEOID'].astype(str)
sc_block_geometry_gdf = sc_block_geometry_gdf.merge(sc_race_df, on='GEOID')
print(sc_block_geometry_gdf.columns)

Index(['GEOID', 'STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry',
       'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K', '60K-100K', '100K_125K',
       '125K_150K', '150K_MORE', 'TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22'],
      dtype='object')


In [18]:
sc_race_gdf = sc_block_geometry_gdf[['GEOID', 'geometry'] + race_columns].copy()
sc_race_gdf['category'] = 'race'

In [19]:
print(sc_race_gdf.columns)

Index(['GEOID', 'geometry', 'TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22', 'category'],
      dtype='object')


In [45]:
print(sc_race_gdf.crs)
print(sc_precincts_gdf.crs)

EPSG:4326
EPSG:4326


In [46]:
print(sc_race_gdf.columns)

Index(['GEOID', 'geometry', 'TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22', 'category'],
      dtype='object')


In [ ]:
variables = [
    'TOT_POP22', 'NHSP_POP22', 'HSP_POP22', 'WHT_NHSP22',
    'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22', 'OTH_NHSP22'
]

In [48]:
sc_precincts_race_gdf = aggregate_racial_data(sc_race_gdf, sc_precincts_gdf, variables)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_96716/2600728937.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_96716/2600728937.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_96716/2600728937.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_96716/2600728937.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead

In [50]:
sc_precincts_race_gdf = sc_precincts_race_gdf.drop(columns=['geometry'])
sc_precincts_race_gdf_json = sc_precincts_race_gdf.to_json(orient='records')

In [51]:
with open('states/south_carolina/demographics/south_carolina_precincts_racial_population.json', 'w') as json_file:
    json.dump(sc_precincts_econ_gdf_json, json_file, indent=4)

## Aggerating block population data to precincts with regiontype

In [25]:
variables = ['TOT_POP22', 'NHSP_POP22', 'HSP_POP22']

In [26]:
sc_precincts_region_gdf = aggregate_region_data(sc_race_gdf, sc_precincts_gdf, variables)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute ins

In [27]:
print(sc_precincts_region_gdf.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'NAME' 'Prec_Code' 'geometry' 'TOT_POP22'
 'NHSP_POP22' 'HSP_POP22' 'region_type']


In [29]:
sc_precincts_gdf = sc_precincts_region_gdf[['UNIQUE_ID', 'COUNTYFP', 'County', 'NAME', 'Prec_Code','region_type', 'geometry']]

In [30]:
print(sc_precincts_gdf.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'NAME' 'Prec_Code' 'region_type'
 'geometry']


In [32]:
print(sc_precincts_gdf.head())

                     UNIQUE_ID COUNTYFP     County             NAME Prec_Code  \
0  ABBEVILLE-:-ABBEVILLE NO. 1      001  ABBEVILLE  ABBEVILLE NO. 1       001   
1  ABBEVILLE-:-ABBEVILLE NO. 2      001  ABBEVILLE  ABBEVILLE NO. 2       002   
2  ABBEVILLE-:-ABBEVILLE NO. 3      001  ABBEVILLE  ABBEVILLE NO. 3       003   
3  ABBEVILLE-:-ABBEVILLE NO. 4      001  ABBEVILLE  ABBEVILLE NO. 4       004   
4       ABBEVILLE-:-ANTREVILLE      001  ABBEVILLE       ANTREVILLE       005   

  region_type                                           geometry  
0       rural  POLYGON ((-82.30056 34.23015, -82.30059 34.230...  
1       rural  POLYGON ((-82.33969 34.18658, -82.33989 34.186...  
2       rural  POLYGON ((-82.37867 34.17718, -82.37873 34.177...  
3       rural  POLYGON ((-82.49060 34.23842, -82.49158 34.238...  
4       rural  POLYGON ((-82.62101 34.28633, -82.62106 34.286...  


## Output final file

In [31]:
sc_precincts_gdf.to_file("states/south_carolina/geodata/south_carolina_precincts.geojson", driver="GeoJSON")

# Aggregate Census Block data to precincts for Maryland:

## Get block level geometry

In [33]:
md_block_inc_gdf = gpd.read_file(
    'raw/census_block/income/md_inc_2022_bg_shape_file/md_inc_2022_bg.shp'
)
md_block_inc_gdf['geometry'] = md_block_inc_gdf ['geometry'].apply(orient_geometry)
md_block_inc_gdf = md_block_inc_gdf .to_crs(epsg=4326)
print(len(md_block_inc_gdf.columns))

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:10: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_72254/3548326586.py:12: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':


24


In [34]:
md_block_geometry_gdf = md_block_inc_gdf[
    ['GEOID','STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry']
]

## Get precincts level geoDataFrames

In [35]:
md_precincts_gdf = gpd.read_file('states/maryland/geodata/maryland_precincts.geojson')
print(len(md_precincts_gdf.columns))

2


## Get Census Block categories csv

In [579]:
md_econ_df = pd.read_csv('processed_individual/md_econ_block.csv')
md_race_df = pd.read_csv('processed_individual/md_race_block.csv')

## Merging census block geometry with economical data

In [580]:
md_econ_df = md_econ_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
md_block_geometry_gdf['GEOID'] = md_block_geometry_gdf['GEOID'].astype(str)
md_econ_df['GEOID'] = md_econ_df['GEOID'].astype(str)
md_block_geometry_gdf = md_block_geometry_gdf.merge(md_econ_df, on='GEOID')
print(md_block_geometry_gdf.columns)

Index(['GEOID', 'STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry',
       'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K', '60K-100K', '100K_125K',
       '125K_150K', '150K_MORE'],
      dtype='object')


In [ ]:
md_economic_gdf = md_block_geometry_gdf[['GEOID', 'geometry'] + economic_columns].copy()
md_economic_gdf['category'] = 'economic'

In [582]:
md_block_geometry_gdf['geometry'] = md_block_geometry_gdf['geometry'].apply(orient_geometry)
final_md_block_gdf = md_block_geometry_gdf.to_crs(epsg=4326)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':


In [ ]:
variables = [
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K',
    '60K-100K', '100K_125K', '125K_150K', '150K_MORE'
]

In [584]:
md_precincts_gdf = aggregate_economic_data(md_economic_gdf, md_precincts_gdf, variables)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif

## Merging census block geometry with racial data

In [585]:
md_race_df = md_race_df.drop(columns=['STATEFP', 'STATE', 'COUNTYFP', 'COUNTY'])
md_block_geometry_gdf['GEOID'] = sc_block_geometry_gdf['GEOID'].astype(str)
md_race_df['GEOID'] = md_race_df['GEOID'].astype(str)
md_block_geometry_gdf = md_block_geometry_gdf.merge(md_race_df, on='GEOID')
print(md_block_geometry_gdf.columns)

Index(['GEOID', 'STATEFP', 'STATE', 'COUNTYFP', 'COUNTY', 'geometry',
       'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K', '60K-100K', '100K_125K',
       '125K_150K', '150K_MORE', 'TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22'],
      dtype='object')


In [586]:
md_race_gdf = md_block_geometry_gdf[['GEOID', 'geometry'] + race_columns].copy()
md_race_gdf['category'] = 'race'

In [587]:
variables = ['TOT_POP22', 'NHSP_POP22', 'HSP_POP22',
       'WHT_NHSP22', 'BLK_NHSP22', 'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22',
       'OTH_NHSP22']

In [588]:
md_precincts_gdf = aggregate_racial_data(md_race_gdf, md_precincts_gdf, variables)

/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:2: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if geom.type == 'Polygon':
/var/folders/3l/c_gyf2kn1vg4ww5h_h4819cm0000gn/T/ipykernel_812/2600728937.py:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  elif geom.type == 'MultiPolygon':


## Export dataframe to GeoJSON

In [589]:
md_precincts_gdf.to_file("states/maryland/geodata/maryland_precincts.geojson", driver="GeoJSON")

## Experimenting Code

In [ ]:
import json
# sc_precincts_gdf = gpd.read_file('states/south_carolina/geodata/south_carolina_precincts.geojson')
# sc_precincts_boundaries = sc_precincts_gdf[["UNIQUE_ID", "geometry"]]
# output_path = 'processed_individual/sc_precinct_boundaries.geojson'
# sc_precincts_boundaries.to_file(output_path, driver="GeoJSON")

# sc_precincts_gdf2 = gpd.read_file('states/south_carolina/geodata/south_carolina_precincts.geojson')
# df_no_geometry = sc_precincts_gdf2.drop(columns=["geometry"])
# output_path = "processed_individual/sc_precinct_nongeo.json"
# df_no_geometry.to_json(output_path, orient="records")

# sc_precincts_racial_groups = pd.read_json('processed_individual/sc_precinct_nongeo.json')
# output_path = 'processed_individual/sc_precinct_racial_groups.json'
# sc_precincts_racial_groups = sc_precincts_racial_groups[["UNIQUE_ID","TOT_POP22","NHSP_POP22","HSP_POP22","WHT_NHSP22","BLK_NHSP22","AIA_NHSP22","ASN_NHSP22","HPI_NHSP22","OTH_NHSP22"]]
# sc_precincts_racial_groups.to_json(output_path, orient="records")

# sc_precincts_boundaries = gpd.read_file('processed_individual/sc_precinct_boundaries.geojson')


# input_path = 'processed_individual/sc_precinct_boundaries.geojson'
# output_path = 'processed_individual/sc_precinct_boundaries_with_bins.geojson'
# bins = [
#     {"bin": 1, "range": [0, 9], "color": [255, 246, 247]},
#     {"bin": 2, "range": [10, 19], "color": [253, 225, 224]},
#     {"bin": 3, "range": [20, 29], "color": [255, 195, 191]},
#     {"bin": 4, "range": [30, 39], "color": [251, 158, 184]},
#     {"bin": 5, "range": [40, 49], "color": [247, 104, 162]},
#     {"bin": 6, "range": [50, 59], "color": [223, 53, 149]},
#     {"bin": 7, "range": [60, 69], "color": [177, 0, 133]},
#     {"bin": 8, "range": [70, 79], "color": [124, 0, 122]},
#     {"bin": 9, "range": [80, 89], "color": [81, 0, 109]},
#     {"bin": 10, "range": [90, 100], "color": [28, 2, 39]}
# ]

# for bin in bins:
#     # Get the RGB values
#     r, g, b = bin["color"]
#     # Convert to hexadecimal and format as #RRGGBB
#     bin["color"] = f"#{r:02x}{g:02x}{b:02x}"



# Step 1: Load the GeoDataFrame from the existing GeoJSON file
# sc_precincts_boundaries = gpd.read_file('processed_individual/sc_precinct_boundaries.geojson')

# geojson = sc_precincts_boundaries.to_json()

# metadata_feature = {
#     "type": "Feature",
#     "geometry": {
#         "type": "Polygon",
#         "coordinates": [
#           [
#             [-0, 0],
#             [-0, 0],
#             [-0, 0],
#             [-0, 0],
#             [-0, 0]
#           ]
#         ]
#     },
#     "properties": {
#         "bins": bins
#     }
# }

# data['features'].append(metadata_feature)


# # Step 4: Save the updated dictionary as a GeoJSON file
# output_path = 'processed_individual/sc_precinct_boundaries_with_bins.geojson'
# with open(output_path, 'w') as f:
#     json.dump(sc_precincts_boundaries, f)
# output_path = 'processed_individual/sc_precinct_boundaries_with_bins.geojson'

# # Load the file and check for the 'bins' key
# with open(output_path, 'r') as f:
#     data = json.load(f)

# # Check if 'bins' exists and print its contents if present
# for bin_info in data["bins"]:
#         print(bin_info)

import matplotlib.colors as mcolors

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file("processed_individual/finishedsc.geojson")

gdf = gdf.drop(2261)
# Check the structure of the GeoDataFrame to ensure 'assigned_color' is present
print(gdf.columns)
for row in gdf.itertuples():
    print(f"row with index {row.Index} has color <{row.assigned_color}>")

# gdf.drop()

# Plot the GeoDataFrame, using 'assigned_color' for coloring
fig, ax = plt.subplots(figsize=(100, 100))

# def hex_to_rgba(hex_color, alpha=1.0):
#     # Convert hex to RGB
#     rgb = mcolors.hex2color(hex_color)
    
#     # Return the RGBA tuple by adding the alpha value
#     return (rgb[0], rgb[1], rgb[2], alpha)

# Customize the plot as needed
ax.set_title("GeoJSON Features Colored by Assigned Color", fontsize=12)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Use the 'assigned_color' property to set the color of each feature
gdf.plot(ax=ax, color=gdf['assigned_color'])

plt.show()





Index(['UNIQUE_ID', 'assigned_color', 'bins', 'geometry'], dtype='object')
row with index 0 has color <#df3595>
row with index 1 has color <#fff6f7>
row with index 2 has color <#fb9eb8>
row with index 3 has color <#df3595>
row with index 4 has color <#7c007a>
row with index 5 has color <#51006d>
row with index 6 has color <#fb9eb8>
row with index 7 has color <#51006d>
row with index 8 has color <#7c007a>
row with index 9 has color <#df3595>
row with index 10 has color <#df3595>
row with index 11 has color <#51006d>
row with index 12 has color <#7c007a>
row with index 13 has color <#b10085>
row with index 14 has color <#df3595>
row with index 15 has color <#51006d>
row with index 16 has color <#fff6f7>
row with index 17 has color <#fff6f7>
row with index 18 has color <#fde1e0>
row with index 19 has color <#7c007a>
row with index 20 has color <#fff6f7>
row with index 21 has color <#51006d>
row with index 22 has color <#7c007a>
row with index 23 has color <#df3595>
row with index 24 has c

ValueError: Invalid RGBA argument: None